I am not competitive in this competition, just trying to learn. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
### Necessary libraries

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
### Seaborn style
sns.set_style("whitegrid")

In [ ]:
### Let's import our data
train_data = pd.read_json('../input/train.json')
### and test if everything OK
train_data.head()

In [ ]:
sns.countplot(train_data.interest_level, order=['low', 'medium', 'high']);
plt.xlabel('Interest Level');
plt.ylabel('Number of occurrences');

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

#df = pd.read_json(open("../input/train.json", "r"))

print(train_data.shape)

In [ ]:
train_data["count_photos"] = train_data["photos"].apply(len)
train_data["count_features"] = train_data["features"].apply(len)
train_data["count_description_words"] = train_data["description"].apply(lambda x: len(x.split(" ")))
train_data["created"] = pd.to_datetime(train_data["created"])
train_data["year"] = train_data["created"].dt.year
train_data["month"] = train_data["created"].dt.month
train_data["day"] = train_data["created"].dt.day
train_data.head()

In [ ]:
important_features = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "count_photos", "count_features", "count_description_words",
             "year", "month", "day"]
X = train_data[important_features]
y = train_data["interest_level"]
X.head()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [ ]:

clf = RandomForestClassifier(n_estimators=1000)
clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
clf.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
eclf2 = VotingClassifier(estimators=[('lr', clf), ('rf', clf2)], voting='soft', weights=[2,1])
eclf2 = eclf2.fit(X_train, y_train)
print(eclf2.predict(X))
y_val_pred = eclf2.predict_proba(X_val)
log_loss(y_val, y_val_pred)


In [ ]:
test_data = pd.read_json(open("../input/test.json", "r"))
print(test_data.shape)
test_data["count_photos"] = test_data["photos"].apply(len)
test_data["count_features"] = test_data["features"].apply(len)
test_data["count_description_words"] = test_data["description"].apply(lambda x: len(x.split(" ")))
test_data["created"] = pd.to_datetime(test_data["created"])
test_data["year"] = test_data["created"].dt.year
test_data["month"] = test_data["created"].dt.month
test_data["day"] = test_data["created"].dt.day
X = test_data[important_features]

y = clf.predict_proba(X)

In [ ]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = test_data["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission.csv", index=False)